# Water table over a multistep impervious base

This is after a problem discussed in an article in Groundwater by Zaidel (https://doi.org/10.1111/gwat.12019):

The Zaidel (2013) paper demonstrates that groundwater flow over a stepped impervious base 
presents significant numerical challenges for MODFLOW. The problem involves an unconfined 
aquifer with abrupt elevation changes in the basal surface, creating a discontinuous 
water table configuration.

Key findings from the paper:

1. **Convergence Issues**: Standard MODFLOW versions (2000, 2005) experienced significant 
    convergence problems when simulating flow over stepped geometries, sometimes failing 
    completely or producing erroneous results.

2. **Drying-Rewetting Challenges**: The transition between saturated and unsaturated 
    conditions at the step boundaries creates numerical instabilities, particularly in 
    thin unconfined aquifers.

3. **Solver Sensitivity**: The paper highlights how solver choice and settings critically 
    impact the ability to successfully simulate these geometries. Different iterative 
    solvers and convergence criteria yield varying degrees of success.

4. **Newton-Raphson Formulation**: Advanced solution schemes like the Newton-Raphson 
    method can improve convergence for these nonlinear problems, though careful selection 
    of solver parameters remains important.

This notebook reproduces the stepped base configuration and explores how different 
MODFLOW 6 settings (confined vs. unconfined, Newton formulation, solver types) affect
convergence behavior and solution quality. But we will start with basic confined flow
and build up from there, tweaking the solver settings and interpreting the output as we go.

## Import packages
Let's start by importing the required packages for building the model and displaying results

In [ ]:
import flopy
from flopy.plot.styles import styles
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil

Give the simualtion a name. It will also be the working directory for all the scenarios below.

In [ ]:
sim_name = "zaidel"

The model represents a 1D cross-section (single row) with 100 columns
Each column is 5 meters wide, creating a 500-meter long profile

In [ ]:

# Model units
length_units = "meters"
time_units = "days"

# Model parameters
nper = 1  # Number of periods
nlay = 1  # Number of layers
nrow = 1  # Number of rows
ncol = 100  # Number of columns
delr = 5.0  # Column width
delc = 1.0  # Row width
top = np.zeros(ncol, dtype=float) + 25.0  # Top of the model
k11 = 0.0001  # Horizontal hydraulic conductivity

Set the starting head. Note that we start with largely saturated cells and the water will consequently drain through the boundary.

In [ ]:
strt = 23.0  # Starting head

Boundary stress period data for the CHD package

In [ ]:
H1 = 23.0  # Constant head in column 1
H2 = 1.0 # Constant head in column <ncol>

# stress period data for CHD
chd_spd = [
        [0, 0, 0, H1],
        [0, 0, ncol - 1, H2],
    ]


We'll use confined flow initially, then progress to unconfined conditions

In [ ]:
# Cell conversion type
icelltype = 0

In [ ]:
# Initial bottom at zero
botm = np.zeros((nlay, nrow, ncol), dtype=float)

# Time discretization
tdis_ds = ((1.0, 1, 1.0),)

In [ ]:
# setup simulation
sim_ws = sim_name
sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")

Create the Iterative Model Solution (IMS) package with rather nominal settings. 
Note that we have `print_option=all` and store the iteration results in `ims.csv`

In [ ]:
nouter = 100
ninner = 50
hclose = 1e-6
rclose = 1e-6
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="all",
    linear_acceleration="cg",
    outer_maximum=nouter,
    outer_dvclose=hclose,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=f"{rclose}",
    csv_inner_output_filerecord="ims.csv",
)

Set up the basic groundwater flow model

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
gwf = flopy.mf6.ModflowGwf(sim, modelname=sim_name, newtonoptions="")
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    icelltype=icelltype,
    k=k11,
    save_flows=True,
    save_specific_discharge=True,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)

Add the output control package for heads and budget

In [ ]:
head_filerecord = f"{sim_name}.hds"
budget_filerecord = f"{sim_name}.cbc"
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=head_filerecord,
    budget_filerecord=budget_filerecord,
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

Next is a utility function to clean up the working directory, write the (updated) simulation to disk, and run (yes, yes, this violates the Single Responsibility Principle, sorry...)

In [ ]:
def save_and_run(sim):
    # save
    shutil.rmtree(sim.sim_path)
    sim.write_simulation(silent=True)

    # run
    result, _ = sim.run_simulation(silent=True)
    if not result:
        print("run failed...")
        return False
    else:
        print("run succeeded...")
        return True

And one more routine to create a plot showing a cross-section of model results as well as the convergence behavior of the numerical solution

In [ ]:


def plot_results(sim, confined=True, show_spdis=False):
    figure_size = (6.3, 5.0)

    with styles.USGSMap():
    
        # plot head
        gwf_model = sim.get_model()

        vmin, vmax = 0, 25
        bot_arr = gwf_model.dis.botm.array
        xedge = gwf_model.modelgrid.xvertices[0]
        zedge = np.array([bot_arr[0, 0, 0]] + bot_arr.flatten().tolist())

        # Create figure for simulation
        extents = (0, ncol * delr, -1, 25.0)
        fig, (ax1, ax2) = plt.subplots(
            2,
            1,
            figsize=figure_size,
            dpi=300,
            constrained_layout=True,
        )

        ax1.set_yscale('linear')

        ax1.set_xlim(extents[:2])
        ax1.set_ylim(extents[2:])

        # model cross section
        fmp = flopy.plot.PlotCrossSection(
            model=gwf_model, ax=ax1, extent=extents, line={"row": 0}
        )
        ax1.fill_between(xedge, zedge, y2=-1, color="0.75", step="pre", lw=0.0)

        # plot head
        head = gwf_model.output.head().get_data()
        wt = head
        if confined:
            wt = None
        pa = fmp.plot_array(head, head=wt, vmin=vmin, vmax=vmax)

        fmp.plot_bc("CHD", color="cyan", head=head)
        ax1.set_xlabel("x-coordinate, in meters")
        ax1.set_ylabel("Elevation, in meters")

        # create legend
        ax1.plot(
            -10000,
            -10000,
            lw=0,
            marker="s",
            ms=10,
            mfc="cyan",
            mec="cyan",
            label="Constant Head",
        )
        ax1.plot(
            -10000,
            -10000,
            lw=0,
            marker="s",
            ms=10,
            mfc="0.75",
            mec="0.75",
            label="Model Base",
        )
        styles.graph_legend(ax1, ncol=2, loc="upper right")

        # plot colorbar
        cax = plt.axes([0.60, 0.82, 0.325, 0.025])
        cbar = plt.colorbar(pa, shrink=0.8, orientation="horizontal", cax=cax)
        cbar.ax.tick_params(size=0)
        cbar.ax.set_xlabel(r"Head, $m$", fontsize=9)

        if show_spdis:
            cbc = gwf_model.output.budget().get_data(text="DATA-SPDIS")[0]
            fmp.plot_vector(cbc["qx"], cbc["qy"], cbc["qz"])

        
        ims_csv = os.path.join(sim.sim_path, "ims.csv")
        df = pd.read_csv(ims_csv)        
        x_ninners = df["total_inner_iterations"].array
        y_dvmax = abs(df["solution_inner_dvmax"].array)      
        ax2.plot(x_ninners, y_dvmax, label="dvmax")
        ax2.set_xlabel("inner iteration number")
        ax2.set_ylabel("dvmax")

        nr_iters = df["total_inner_iterations"].array[-1]
        min_y = 10e-2 * min(y_dvmax)
        max_y = 10e+3 * max(y_dvmax)

        # mark next outer
        nouter_diff = [int(df["nouter"][i + 1] - df["nouter"][i]) for i in range(df["nouter"].size - 1)]
        new_outers = np.where(np.asarray(nouter_diff) == 1)[0] + 1
        for idx_outer in new_outers:
            ax2.vlines(idx_outer + 0.5, min_y, max_y, color='grey', linestyle='dashed', linewidth=0.8)
        ax2.hlines(hclose, 1, nr_iters, color='darkgreen', label="dvclose")

        ax2.set_ylim(min_y, max_y)
        ax2.set_yscale("log")
        ax2.xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
        styles.graph_legend(ax2, ncol=2, loc="upper right")

        # ax2outer = ax2.twiny()
        # ax2outer.spines['bottom'].set_position(('outward', 36))
        # ax2outer.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
        # ax2outer.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
        # ax2outer.set_xlabel("outer iteration number")

        # ax2outer.plot()

        plt.show()

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim)

In [ ]:
# add staircase bottom
dis = sim.get_model().get_package("DIS")
nsteps = 5
for j in range(ncol):
    dis.botm[:, :, j] = 20.0 - 5.0 * (nsteps*j//ncol)

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim, False)

In [ ]:
# run unconfined
sim.get_model().npf.icelltype = 1

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim, False)

In [ ]:
# activate newton formulation
sim.get_model().newtonoptions = "newton"

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim, False)

In [ ]:
# set complexity to complex (and unset CG)
sim.ims.linear_acceleration = None
sim.ims.complexity = "complex"

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim, False)

In [ ]:
# set complexity to complex (and unset CG)
sim.ims.linear_acceleration = "bicgstab"
sim.ims.complexity = None

In [ ]:
success = save_and_run(sim)
if success:
    plot_results(sim, False)